# F1 Elo Prototyping

Notebook for exploring ideas before binning or porting to scripts.

## Environment setup

In [1]:
import yaml

import pandas as pd
import plotly.express as px

In [2]:
# Move project root folder
%cd ..

/Users/mwtmurphy/projects/f1-elo


/Users/mwtmurphy/Library/Caches/pypoetry/virtualenvs/f1-elo-tRE7S7Hg-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Data importing

In [20]:
with open("params.yaml") as conf_file:
    CONFIG = yaml.safe_load(conf_file)

mod_df = pd.read_csv(CONFIG["data"]["modelled_path"])
constructors_df = pd.read_csv(CONFIG["data"]["constructors_csv"])[["constructorId", "name"]].rename(columns={"name": "constructorName"})
drivers_df = pd.read_csv(CONFIG["data"]["drivers_csv"])[["driverId", "driverRef"]]

## Data visualisation

In [21]:
con_df = mod_df.groupby(["year", "date", "constructorId"])["constructorScore"].max().reset_index()
con_df = con_df.merge(constructors_df, on="constructorId", how="left")
cur_con = set(con_df.loc[con_df["year"] == 2024, "constructorId"])
cur_df = con_df[con_df["constructorId"].isin(cur_con) & (con_df["year"] > 2010)]

px.line(cur_df, x="date", y="constructorScore", color="constructorName")

In [22]:
cur_df.loc[(cur_df["date"] == cur_df["date"].max()), ["constructorName", "constructorScore"]].sort_values("constructorScore", ascending=False).drop_duplicates()

,constructorName,constructorScore
12871,Red Bull,1796.971819
12870,Ferrari,1726.404103
12874,Mercedes,1701.023825
12868,McLaren,1685.843528
12873,Aston Martin,1572.046843
12872,Sauber,1548.079025
12877,RB F1 Team,1518.244900
12876,Alpine F1 Team,1490.258123
12875,Haas F1 Team,1485.018449
12869,Williams,1458.211512


In [24]:
dri_df = mod_df.groupby(["year", "date", "driverId"])["driverScore"].max().reset_index()
dri_df = dri_df.merge(drivers_df, on="driverId", how="left")
cur_dri = set(dri_df.loc[dri_df["year"] == 2024, "driverId"])
cur_dri_df = dri_df[dri_df["driverId"].isin(cur_dri) & (dri_df["year"] > 2010)]

px.line(cur_dri_df, x="date", y="driverScore", color="driverRef")

In [25]:
cur_dri_df.loc[(cur_dri_df["date"] == cur_dri_df["date"].max()), ["driverRef", "driverScore"]].sort_values("driverScore", ascending=False).drop_duplicates()

,driverRef,driverScore
26355,max_verstappen,1961.950539
26361,norris,1815.509350
26349,alonso,1751.968031
26360,leclerc,1739.437965
26348,hamilton,1688.052404
26356,sainz,1662.909169
26366,piastri,1641.439192
26357,ocon,1630.923083
26362,russell,1616.181912
26359,gasly,1607.877850


# Modelling optimisation

In [ ]:
# runtime improvement per model run: 32.4s -> 2.89s